<a href="https://colab.research.google.com/github/CyanideBoy/word2vec-training/blob/master/word2vec_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x
import numpy as np
np.random.seed(13)

import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

import gensim

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
path = get_file('alice.txt', origin='http://www.gutenberg.org/files/11/11-0.txt')
corpus = open(path).readlines()[:300]
corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)

nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1
dim = 100
window_size = 2

In [3]:
print(corpus)
print(len(corpus))  #Nb sent
print(nb_samples)   #Nb Words
print(V)            #Nb distinct words

[[1, 141, 105, 85, 8, 55, 106, 10, 107, 43, 142, 143], [24, 85, 61, 16, 1, 62, 8, 321, 322, 26, 36, 323, 4, 30], [144, 36, 324, 325, 17, 326, 327, 5, 187, 5, 188, 31], [328, 62, 5, 108, 1, 329, 8, 1, 141, 105, 330, 331], [30, 24, 85, 31, 332, 26, 333, 105, 334], [335, 55, 106, 10, 107], [336, 142, 143], [337, 338, 339, 340, 341, 85, 342], [343, 344, 345, 346, 347], [348, 189, 349, 350, 351], [352, 8, 24, 141, 105, 85, 55, 106, 10, 107], [353, 43, 354, 355, 4, 356, 357], [55, 106, 10, 107], [43, 142, 143], [1, 358, 359, 360, 361, 362], [37, 11, 15, 1, 40, 86], [37, 363, 1, 190, 8, 145], [37, 364, 6, 365, 366, 4, 6, 109, 367], [37, 368, 1, 40, 369, 10, 6, 21, 370], [37, 371, 191, 110, 6, 372], [37, 373, 374, 4, 375], [37, 376, 6, 377, 192, 378], [37, 379, 1, 380, 193, 381], [37, 382, 1, 383, 384, 385], [37, 386, 1, 387, 388], [37, 389, 146, 390, 1, 391], [37, 392, 55, 393], [15, 1, 40, 86], [12, 7, 394, 3, 63, 18, 194, 8, 395, 43, 13, 195, 22, 1], [396, 4, 8, 147, 64, 3, 65, 87, 31, 397,

In [0]:
def generate_data(corpus, window_size, V):
    maxlen = window_size*2
    for words in corpus:  #words = sentence
        L = len(words)
        for index, word in enumerate(words):
            contexts = []
            labels   = []            
            s = index - window_size
            e = index + window_size + 1
            
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)

            #x = contexts
            #y = labels
            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            y = np_utils.to_categorical(labels, V)
            yield (x, y)

In [5]:
'''
flag= 2
for x in generate_data(corpus,window_size, V):
  if flag >0:
    print(x)
    flag -= 1
    print("_____")
'''

'\nflag= 2\nfor x in generate_data(corpus,window_size, V):\n  if flag >0:\n    print(x)\n    flag -= 1\n    print("_____")\n'

In [6]:
for x in range(-2,3):
  print(x)

-2
-1
0
1
2


In [7]:
cbow = Sequential()
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
cbow.add(Dense(V, activation='softmax'))
cbow.summary()




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4, 100)            75900     
_________________________________________________________________
lambda_1 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 759)               76659     
Total params: 152,559
Trainable params: 152,559
Non-trainable params: 0
_________________________________________________________________


In [8]:


cbow.compile(loss='categorical_crossentropy', optimizer='adadelta')



In [9]:


for ite in range(10):
    loss = 0.
    for x, y in generate_data(corpus, window_size, V):
        loss += cbow.train_on_batch(x, y)

    print(ite, loss)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








0 16395.747025966644
1 15214.69013261795
2 15067.680879354477
3 14984.98672246933
4 14882.560213446617
5 14792.352047085762
6 14705.268415004015
7 14622.229464113712
8 14541.870840251446
9 14462.75509737432


In [10]:
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))

8

In [0]:
vectors = cbow.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()


In [12]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [13]:
w2v.most_similar(positive=['the'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('this', 0.6630313396453857),
 ('nothing', 0.6297290325164795),
 ('out', 0.5834192037582397),
 ('a', 0.5582808256149292),
 ('its', 0.5288040637969971),
 ('“do', 0.5223385095596313),
 ('ever', 0.521538496017456),
 ('latitude', 0.5179874897003174),
 ('things', 0.5157833099365234),
 ('beautifully', 0.5155222415924072)]

In [14]:
w2v.most_similar(positive=['alice'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('she', 0.7184145450592041),
 ('ever', 0.5955518484115601),
 ('going', 0.5869276523590088),
 ('that', 0.5822823643684387),
 ('thought', 0.5785404443740845),
 ('you', 0.5774433612823486),
 ('them', 0.5591905117034912),
 ('no', 0.5519707202911377),
 ('suddenly', 0.5451568365097046),
 ('it', 0.5431429147720337)]